# data_wrangling.ipynb (Download the data to your local machine to run it for 2017 and 2017. Link Provided below)
# by Tarak Patel


### 1.   This script uses the downloaded airlines data from Kaggle (2017 and 2018) 
####     https://www.kaggle.com/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018/version/1?select=2017.csv
####     https://www.kaggle.com/yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018/version/1?select=2018.csv
### 2.   I uploaded the data to aws S3 and successfully pulled the data from AWS in this file but we maxed out the data transfer available for Free tier
###      (15gb), so we had to use the data from our local drive to run results. 
### 3.   We also cannot upload the data to our github because it also has a free tier limit. 
### 4.   Please Download the data to your local machine to run it for 2017 and 2018. 
### 5.   I cleaned the data using Pandas.
### 6.   The cleaned data is uploaded to "data_manipulation_modeling/data_clean" folder in github repository. The three CSV files are named 
###      2016_TUS, 2017_TUS and 2018_TUS
### 7.   The 2017_TUS is used to train the Machine learning Model
### 8.   The 2018_TUS is used to test the Machine Learning Model.
### 9.   The 2018_TUS is also used by I and the team members to create dashboard in Tableau. 
### 10.  We decided to create a website and upload the Tableau dashboards to the website. 
### 11.  We used github to host the website with dashboards. Link is provided separately with the homework.


In [1]:
# Import Dependencies
import pandas as pd 
import json
import requests
import numpy as np
import datetime
from datetime import timedelta
import math


In [2]:
# I uploaded the data to aws S3 and successfully pulled the data from AWS in this file but we maxed out the data transfer available for Free tier
# (15gb), So we had to use the data from our local drive to run results. 
# We also cannot upload the data to our github because it also has a free tier limit. 
# Please use the links to download data from Kaggle and use it from your desktop

# Read the CSV file from AWS to Pandas Dataframe
url = "C:/Users/tarak/Desktop/Final Project/Resources/2018.csv"

df = pd.read_csv(url)

df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,...,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,...,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,...,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,...,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,...,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Filter the data for Tucson Airport
df1 = df.loc[df['ORIGIN'] == 'TUS']


# Remove the Unnamed columns which as no values
tus_df = df1.drop(['Unnamed: 27'], axis = 1)
tus_df

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
421,2018-01-01,UA,1769,TUS,DEN,1440,1454.0,14.0,12.0,1506.0,...,0.0,122.0,105.0,89.0,639.0,NaN,NaN,NaN,NaN,NaN
1470,2018-01-01,UA,288,TUS,IAH,730,724.0,-6.0,11.0,735.0,...,0.0,144.0,125.0,109.0,936.0,NaN,NaN,NaN,NaN,NaN
1916,2018-01-01,AS,641,TUS,SEA,1258,1300.0,2.0,11.0,1311.0,...,0.0,201.0,188.0,170.0,1216.0,NaN,NaN,NaN,NaN,NaN
3846,2018-01-01,EV,4403,TUS,IAH,1208,1145.0,-23.0,10.0,1155.0,...,0.0,148.0,133.0,109.0,936.0,NaN,NaN,NaN,NaN,NaN
6720,2018-01-01,OO,3012,TUS,LAX,800,757.0,-3.0,11.0,808.0,...,0.0,121.0,118.0,71.0,451.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7212416,2018-12-31,OO,5923,TUS,DEN,815,825.0,10.0,10.0,835.0,...,0.0,130.0,123.0,86.0,639.0,NaN,NaN,NaN,NaN,NaN
7212429,2018-12-31,OO,5945,TUS,SFO,659,653.0,-6.0,15.0,708.0,...,0.0,146.0,136.0,115.0,751.0,NaN,NaN,NaN,NaN,NaN
7212747,2018-12-31,AA,137,TUS,DFW,624,615.0,-9.0,10.0,625.0,...,0.0,136.0,119.0,95.0,813.0,NaN,NaN,NaN,NaN,NaN
7213166,2018-12-31,AA,539,TUS,LAX,1159,1339.0,100.0,11.0,1350.0,...,0.0,101.0,90.0,75.0,451.0,0.0,0.0,0.0,0.0,89.0


In [4]:
# Create a Function that converts time columns into Actual Time format (hh:mm)

def time_conv(mil_time):
  hh_mm = []
  for time in mil_time:
    if time == 2400:
      time = 0
    if math.isnan(time):
      hh_mm.append(time)
    elif time < 59:
      hh_mm.append(datetime.datetime.strptime(str(int(time)), '%M').strftime('%H:%M'))  
    else:
      hh_mm.append(datetime.datetime.strptime(str(int(time)), '%H%M').strftime('%H:%M'))  
  return hh_mm


In [6]:
# Use the above function to convert the time columns to hh:mm and create new columns in the tus_df for each converted column

tus_df['CRS_DEP_HM'] = time_conv(tus_df['CRS_DEP_TIME'])
tus_df['DEP_TIME_HM'] = time_conv(tus_df['DEP_TIME'])
tus_df['WHEELS_OFF_HM'] = time_conv(tus_df['WHEELS_OFF'])
tus_df['WHEELS_ON_HM'] = time_conv(tus_df['WHEELS_ON'])
tus_df['CRS_ARR_TIME_HM'] = time_conv(tus_df['CRS_ARR_TIME'])
tus_df['ARR_TIME_HM'] = time_conv(tus_df['ARR_TIME'])

tus_df.head()


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM
421,2018-01-01,UA,1769,TUS,DEN,1440,1454.0,14.0,12.0,1506.0,...,NaN,NaN,NaN,NaN,14:40,14:54,15:06,16:35,16:42,16:39
1470,2018-01-01,UA,288,TUS,IAH,730,724.0,-6.0,11.0,735.0,...,NaN,NaN,NaN,NaN,07:30,07:24,07:35,10:24,10:54,10:29
1916,2018-01-01,AS,641,TUS,SEA,1258,1300.0,2.0,11.0,1311.0,...,NaN,NaN,NaN,NaN,12:58,13:00,13:11,15:01,15:19,15:08
3846,2018-01-01,EV,4403,TUS,IAH,1208,1145.0,-23.0,10.0,1155.0,...,NaN,NaN,NaN,NaN,12:08,11:45,11:55,14:44,15:36,14:58
6720,2018-01-01,OO,3012,TUS,LAX,800,757.0,-3.0,11.0,808.0,...,NaN,NaN,NaN,NaN,08:00,07:57,08:08,08:19,09:01,08:55


In [7]:
# Add Day of the week to tus_df

tus_df['FL_DATE'] = pd.to_datetime(tus_df['FL_DATE'], format = '%Y/%m/%d')
tus_df['day_of_week'] = tus_df['FL_DATE'].dt.day_name()
tus_df.head()


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week
421,2018-01-01,UA,1769,TUS,DEN,1440,1454.0,14.0,12.0,1506.0,...,NaN,NaN,NaN,14:40,14:54,15:06,16:35,16:42,16:39,Monday
1470,2018-01-01,UA,288,TUS,IAH,730,724.0,-6.0,11.0,735.0,...,NaN,NaN,NaN,07:30,07:24,07:35,10:24,10:54,10:29,Monday
1916,2018-01-01,AS,641,TUS,SEA,1258,1300.0,2.0,11.0,1311.0,...,NaN,NaN,NaN,12:58,13:00,13:11,15:01,15:19,15:08,Monday
3846,2018-01-01,EV,4403,TUS,IAH,1208,1145.0,-23.0,10.0,1155.0,...,NaN,NaN,NaN,12:08,11:45,11:55,14:44,15:36,14:58,Monday
6720,2018-01-01,OO,3012,TUS,LAX,800,757.0,-3.0,11.0,808.0,...,NaN,NaN,NaN,08:00,07:57,08:08,08:19,09:01,08:55,Monday


In [8]:
# Airport Codes file is located in the "data_manipulation_modeling/data__raw"
url_airport = "../data__raw/airports_codes.csv"

airport = pd.read_csv(url_airport)

airport.head()

,AirportCode,CityName,WorldAreaCode
0,AAC,"AL ARISH, ARAB REP. OF EGYPT",591
1,AAE,"ANNABA, ALGERIA",500
2,AAF,"APALACHICOLA, FLORIDA, USA",33
3,AAH,"AACHEN, GERMANY",429
4,AAI,"ARRAIAS, BRAZIL",316


In [9]:
# Merge the airport dataframe with tus_df to get the City Names for Destination City.

merge_dest_df = pd.merge(airport, tus_df, left_on='AirportCode', right_on='DEST', how='inner')

merge_dest_df.head()

,AirportCode,CityName,WorldAreaCode,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,...,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week
0,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-01,DL,1240,TUS,ATL,1453,1600.0,...,0.0,0.0,55.0,14:53,16:00,16:27,21:18,20:29,21:24,Monday
1,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-01,DL,1345,TUS,ATL,710,705.0,...,NaN,NaN,NaN,07:10,07:05,07:15,12:10,12:39,12:14,Monday
2,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-02,DL,1240,TUS,ATL,1453,1446.0,...,NaN,NaN,NaN,14:53,14:46,15:03,19:54,20:29,20:04,Tuesday
3,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-02,DL,1345,TUS,ATL,710,703.0,...,NaN,NaN,NaN,07:10,07:03,07:14,12:04,12:39,12:09,Tuesday
4,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-03,DL,1240,TUS,ATL,1340,1335.0,...,NaN,NaN,NaN,13:40,13:35,13:47,18:42,19:13,18:52,Wednesday


In [10]:
# Merge the merge_dest_df with airport dataframe to get the City Names for Arrival City.
merge_dest_origin_df = pd.merge(airport, merge_dest_df, left_on='AirportCode', right_on='ORIGIN', how='inner')
merge_dest_origin_df.head()

,AirportCode_x,CityName_x,WorldAreaCode_x,AirportCode_y,CityName_y,WorldAreaCode_y,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,...,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week
0,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-01,DL,1240,TUS,...,0.0,0.0,55.0,14:53,16:00,16:27,21:18,20:29,21:24,Monday
1,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-01,DL,1345,TUS,...,NaN,NaN,NaN,07:10,07:05,07:15,12:10,12:39,12:14,Monday
2,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-02,DL,1240,TUS,...,NaN,NaN,NaN,14:53,14:46,15:03,19:54,20:29,20:04,Tuesday
3,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-02,DL,1345,TUS,...,NaN,NaN,NaN,07:10,07:03,07:14,12:04,12:39,12:09,Tuesday
4,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-03,DL,1240,TUS,...,NaN,NaN,NaN,13:40,13:35,13:47,18:42,19:13,18:52,Wednesday


In [11]:
# Rename Columns City Columns in merge_dest_origin_df and create a new Dataframe

rename_dest_origin_df = merge_dest_origin_df.rename(columns = {'CityName_x': 'origin_city_name',
                             'CityName_y': 'dest_city_name'})

rename_dest_origin_df.head()

,AirportCode_x,origin_city_name,WorldAreaCode_x,AirportCode_y,dest_city_name,WorldAreaCode_y,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,...,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week
0,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-01,DL,1240,TUS,...,0.0,0.0,55.0,14:53,16:00,16:27,21:18,20:29,21:24,Monday
1,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-01,DL,1345,TUS,...,NaN,NaN,NaN,07:10,07:05,07:15,12:10,12:39,12:14,Monday
2,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-02,DL,1240,TUS,...,NaN,NaN,NaN,14:53,14:46,15:03,19:54,20:29,20:04,Tuesday
3,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-02,DL,1345,TUS,...,NaN,NaN,NaN,07:10,07:03,07:14,12:04,12:39,12:09,Tuesday
4,TUS,"TUCSON, ARIZONA, USA",81,ATL,"ATLANTA, GEORGIA, USA",34,2018-01-03,DL,1240,TUS,...,NaN,NaN,NaN,13:40,13:35,13:47,18:42,19:13,18:52,Wednesday


In [12]:
# Drop colums not needed in the final Database
tus_final_df = rename_dest_origin_df.drop(['AirportCode_x', 'WorldAreaCode_x', 'AirportCode_y', 'WorldAreaCode_y'], axis = 1)
tus_final_df.head()

,origin_city_name,dest_city_name,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week
0,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-01,DL,1240,TUS,ATL,1453,1600.0,67.0,...,0.0,0.0,55.0,14:53,16:00,16:27,21:18,20:29,21:24,Monday
1,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-01,DL,1345,TUS,ATL,710,705.0,-5.0,...,NaN,NaN,NaN,07:10,07:05,07:15,12:10,12:39,12:14,Monday
2,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-02,DL,1240,TUS,ATL,1453,1446.0,-7.0,...,NaN,NaN,NaN,14:53,14:46,15:03,19:54,20:29,20:04,Tuesday
3,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-02,DL,1345,TUS,ATL,710,703.0,-7.0,...,NaN,NaN,NaN,07:10,07:03,07:14,12:04,12:39,12:09,Tuesday
4,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-03,DL,1240,TUS,ATL,1340,1335.0,-5.0,...,NaN,NaN,NaN,13:40,13:35,13:47,18:42,19:13,18:52,Wednesday


In [13]:
# Create a new "Delay" column in tus_final_df.
# with the value of 0 if the delay is less than 30 minutes 
# and 1 if the value in Arr_delay column is Greater than 30 Minutes
DELAY = []
for delay in tus_final_df['ARR_DELAY']:
  if delay < 30:
    DELAY.append(0)
  else:
    DELAY.append(1)

tus_final_df['DELAY'] = DELAY

tus_final_df.head()

,origin_city_name,dest_city_name,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week,DELAY
0,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-01,DL,1240,TUS,ATL,1453,1600.0,67.0,...,0.0,55.0,14:53,16:00,16:27,21:18,20:29,21:24,Monday,1
1,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-01,DL,1345,TUS,ATL,710,705.0,-5.0,...,NaN,NaN,07:10,07:05,07:15,12:10,12:39,12:14,Monday,0
2,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-02,DL,1240,TUS,ATL,1453,1446.0,-7.0,...,NaN,NaN,14:53,14:46,15:03,19:54,20:29,20:04,Tuesday,0
3,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-02,DL,1345,TUS,ATL,710,703.0,-7.0,...,NaN,NaN,07:10,07:03,07:14,12:04,12:39,12:09,Tuesday,0
4,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-03,DL,1240,TUS,ATL,1340,1335.0,-5.0,...,NaN,NaN,13:40,13:35,13:47,18:42,19:13,18:52,Wednesday,0


In [14]:
# Export the final tus_final_df to CSV file for 2018 and repeat the same steps to export 2017_TUS file
# rename the file here so it doesnot overide the 2018 file. 
tus_final_df.to_csv('2018_TUS.csv', index = False)
# tus_final_df.to_csv('2017_TUS.csv', index = False)
# tus_final_df.to_csv('2016_TUS.csv', index = False)

tus_final_df.head()

,origin_city_name,dest_city_name,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,...,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,CRS_DEP_HM,DEP_TIME_HM,WHEELS_OFF_HM,WHEELS_ON_HM,CRS_ARR_TIME_HM,ARR_TIME_HM,day_of_week,DELAY
0,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-01,DL,1240,TUS,ATL,1453,1600.0,67.0,...,0.0,55.0,14:53,16:00,16:27,21:18,20:29,21:24,Monday,1
1,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-01,DL,1345,TUS,ATL,710,705.0,-5.0,...,NaN,NaN,07:10,07:05,07:15,12:10,12:39,12:14,Monday,0
2,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-02,DL,1240,TUS,ATL,1453,1446.0,-7.0,...,NaN,NaN,14:53,14:46,15:03,19:54,20:29,20:04,Tuesday,0
3,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-02,DL,1345,TUS,ATL,710,703.0,-7.0,...,NaN,NaN,07:10,07:03,07:14,12:04,12:39,12:09,Tuesday,0
4,"TUCSON, ARIZONA, USA","ATLANTA, GEORGIA, USA",2018-01-03,DL,1240,TUS,ATL,1340,1335.0,-5.0,...,NaN,NaN,13:40,13:35,13:47,18:42,19:13,18:52,Wednesday,0
